In [36]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

#params = {
#    'task': 'train',
#    'boosting_type': 'gbdt',
#    'objective': 'binary',
#    'metric': {'auc'},
#    'num_leaves': 60,
#    'min_data_in_leaf': 40,
#    'feature_fraction': 0.6,
#    'bagging_fraction': 0.96,
#    'bagging_freq': 1,
#    'verbose': 0
#}

params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 400,
    'min_data_in_leaf': 2,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'min_gain_to_split': 0,
    'verbose': 0
}

In [10]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/latest/"

In [24]:
def training(train, valid):
    X = lgb.Dataset(train.drop(['user_id', 'work_id', 'rating'], axis=1), train['rating'])
    V = lgb.Dataset(valid.drop(['user_id', 'work_id', 'rating'], axis=1), valid['rating'], reference=X)
    gbdt = lgb.train(params, X, valid_sets=[X,V], num_boost_round=200, early_stopping_rounds=20, verbose_eval=False)
    Yvp = gbdt.predict(valid.drop(['user_id', 'work_id', 'rating'], axis=1), num_iteration=gbdt.best_iteration)
    Ytp = gbdt.predict(train.drop(['user_id', 'work_id', 'rating'], axis=1), num_iteration=gbdt.best_iteration)
    return (roc_auc_score(train['rating'].values, Ytp), roc_auc_score(valid['rating'].values, Yvp))

In [11]:
def cv(param, paramlst):
    trainauc = [0.0]*len(paramlst)
    validauc = [0.0]*len(paramlst)
    for i, p in enumerate(paramlst):
        params[param]=p
        tv = [0,0,0]
        vv = [0,0,0]
        for fold in [1,2,3]:
            t = pd.read_csv(data+'train_{0}.csv'.format(str(fold)))
            v = pd.read_csv(data+'valid_{0}.csv'.format(str(fold)))
            tv[fold-1], vv[fold-1] = training(t, v)
        trainauc[i]=np.mean(tv)
        validauc[i]=np.mean(vv)
    paramtable = pd.DataFrame({
        'TrainingSet': trainauc,
        'ValidationSet': validauc
    }, columns=['TrainingSet', 'ValidationSet'], index=pd.Index(paramlst, name=param))
    print(paramtable)

In [34]:
cv('bagging_fraction', [0.4, 0.5, 0.6, 0.7, 0.8])

                  TrainingSet  ValidationSet
bagging_fraction                            
0.4                  0.990154       0.789215
0.5                  0.991352       0.792540
0.6                  0.990290       0.795284
0.7                  0.989913       0.796650
0.8                  0.989267       0.798472


In [35]:
import gc

gc.enable()

In [37]:
gc.collect()

458

LibFM

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

path = "/mnt/d/Data/mangaki-data-challenge/baseline/"

In [7]:
t = []
for cv in [1, 2, 3]:
    v = pd.read_csv(path+"valid_{0}.csv".format(cv))
    pv = pd.read_csv(path+"../libfm/valid{0}_result.csv".format(cv), header=None, names=['prob'])
    t.append(roc_auc_score(v.rating, pv.values))
print(np.mean(t))

0.802645262543


In [5]:
pv

,prob
0,0.507876
1,0.537239
2,0.638249
3,0.534933
4,0.296849
5,0.350443
6,0.174399
7,0.631326
8,0.488645
9,0.450577
